In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install -U sentencepiece
!pip install -U urllib3
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 57.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none

In [2]:
from datasets import load_dataset
from transformers import pipeline

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

In [3]:
device = 'gpu'
model_ckpt = 'facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [4]:
samsum = load_dataset('samsum')
samsum

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [5]:
samsum['train'][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [6]:
samsum['test'][0]

{'id': '13862856',
 'dialogue': "Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye",
 'summary': "Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."}

In [7]:
samsum['validation'][0]

{'id': '13817023',
 'dialogue': "A: Hi Tom, are you busy tomorrow’s afternoon?\r\nB: I’m pretty sure I am. What’s up?\r\nA: Can you go with me to the animal shelter?.\r\nB: What do you want to do?\r\nA: I want to get a puppy for my son.\r\nB: That will make him so happy.\r\nA: Yeah, we’ve discussed it many times. I think he’s ready now.\r\nB: That’s good. Raising a dog is a tough issue. Like having a baby ;-) \r\nA: I'll get him one of those little dogs.\r\nB: One that won't grow up too big;-)\r\nA: And eat too much;-))\r\nB: Do you know which one he would like?\r\nA: Oh, yes, I took him there last Monday. He showed me one that he really liked.\r\nB: I bet you had to drag him away.\r\nA: He wanted to take it home right away ;-).\r\nB: I wonder what he'll name it.\r\nA: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))",
 'summary': 'A will go to the animal shelter tomorrow to get a puppy for her son. They already visited the shelter last Monday an

In [8]:
dialogue_len = [len(x['dialogue'].split()) for x in samsum['train']]
summary_len = [len(x['summary'].split()) for x in samsum['train']]

In [9]:
max_dialogue_len=max(dialogue_len)
max_summary_len=max(summary_len)
# max no. of tokens in dialogue
max_dialogue_len


803

In [10]:
# max no. of tokens in summary
max_summary_len

64

In [11]:
def get_feature(batch):
  encodings = tokenizer(batch['dialogue'], text_target=batch['summary'],
                        max_length=1024, truncation=True)

  encodings = {'input_ids': encodings['input_ids'],
               'attention_mask': encodings['attention_mask'],
               'labels': encodings['labels']}

  return encodings

In [12]:
samsum_pt = samsum.map(get_feature, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [13]:
samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [14]:
columns = ['input_ids', 'labels', 'attention_mask']
samsum_pt.set_format(type='torch', columns=columns)

In [15]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = 'bart_samsum',
    num_train_epochs=1,
    warmup_steps = 500,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset = samsum_pt['train'], eval_dataset = samsum_pt['validation'])


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=230, training_loss=1.4833635910697605, metrics={'train_runtime': 1775.705, 'train_samples_per_second': 8.296, 'train_steps_per_second': 0.13, 'total_flos': 8823027777404928.0, 'train_loss': 1.4833635910697605, 'epoch': 0.9991854466467553})

In [18]:
trainer.save_model('bart_samsum_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [29]:
pipe = pipeline('summarization', model='bart_samsum_model')
gen_kwargs = {'length_penalty': 0.8, 'num_beams': 8}

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Custom dialouge

In [ ]:
dialogue="""
Alice: Hi Bob, I’m interested in the new smartphone you have on sale. Can you tell me more about its features?

Bob: Absolutely, Alice. The new smartphone has a 6.7-inch display, 128GB storage, and a 48MP camera. It also supports 5G and comes with a long-lasting battery.

Alice: That sounds great. Does it come with any warranty?

Bob: Yes, it comes with a one-year manufacturer’s warranty that covers any defects.

Alice: Perfect. Can you also tell me about the return policy?

Bob: Sure. We have a 30-day return policy, so if you're not satisfied, you can return the phone within 30 days for a full refund.

Alice: Thanks, Bob. I think I’ll go ahead and purchase it."""
summary=pipe(dialogue, **gen_kwargs)
summary

[{'summary_text': "Alice is interested in the new smartphone Bob has on sale. The new smartphone has a 6.7-inch display, 128GB storage, and a 48MP camera. It also supports 5G and comes with a long-lasting battery. It has a one-year manufacturer's warranty and a 30-day return policy."}]

In [20]:
!zip bart_samsum.zip -r bart_samsum_model/

  adding: bart_samsum_model/ (stored 0%)
  adding: bart_samsum_model/special_tokens_map.json (deflated 52%)
  adding: bart_samsum_model/vocab.json (deflated 59%)
  adding: bart_samsum_model/generation_config.json (deflated 47%)
  adding: bart_samsum_model/config.json (deflated 61%)
  adding: bart_samsum_model/tokenizer_config.json (deflated 75%)
  adding: bart_samsum_model/model.safetensors (deflated 7%)
  adding: bart_samsum_model/tokenizer.json (deflated 72%)
  adding: bart_samsum_model/training_args.bin (deflated 51%)
  adding: bart_samsum_model/merges.txt (deflated 53%)


In [23]:
!pip install transformers rouge-score numpy

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d230f712fde3a6819a5ee918e8446d4ba5b3bdf3872232c68ac94008caa18beb
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [24]:
from transformers import pipeline
from rouge_score import rouge_scorer
import numpy as np

In [35]:
test_data = samsum['test'].select(range(100))

In [36]:
# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to generate summaries and compute ROUGE scores with dynamic length adjustment
def evaluate_summarizer(test_data, gen_kwargs):
    rouge1_scores = {'precision': [], 'recall': [], 'fmeasure': []}
    rouge2_scores = {'precision': [], 'recall': [], 'fmeasure': []}
    rougeL_scores = {'precision': [], 'recall': [], 'fmeasure': []}

    for data in test_data:
        dialogue = data['dialogue']
        reference_summary = data['summary']

        # Calculate the length of the reference summary
        ref_length = len(reference_summary.split())

        # Adjust gen_kwargs with dynamic length constraints
        gen_kwargs["max_length"] = ref_length + 7
        gen_kwargs["min_length"] = max(5, ref_length - 7)

        # Generate summary with dynamic length constraints
        generated_summary = pipe(dialogue, **gen_kwargs)
        generated_summary_text = generated_summary[0]['summary_text']

        # Compute ROUGE scores
        scores = scorer.score(reference_summary, generated_summary_text)
        rouge1_scores['precision'].append(scores['rouge1'].precision)
        rouge1_scores['recall'].append(scores['rouge1'].recall)
        rouge1_scores['fmeasure'].append(scores['rouge1'].fmeasure)

        rouge2_scores['precision'].append(scores['rouge2'].precision)
        rouge2_scores['recall'].append(scores['rouge2'].recall)
        rouge2_scores['fmeasure'].append(scores['rouge2'].fmeasure)

        rougeL_scores['precision'].append(scores['rougeL'].precision)
        rougeL_scores['recall'].append(scores['rougeL'].recall)
        rougeL_scores['fmeasure'].append(scores['rougeL'].fmeasure)

    # Compute average ROUGE scores
    avg_rouge1 = {key: np.mean(val) for key, val in rouge1_scores.items()}
    avg_rouge2 = {key: np.mean(val) for key, val in rouge2_scores.items()}
    avg_rougeL = {key: np.mean(val) for key, val in rougeL_scores.items()}

    return avg_rouge1, avg_rouge2, avg_rougeL

# Evaluate the summarizer with dynamic length adjustment
avg_rouge1, avg_rouge2, avg_rougeL = evaluate_summarizer(test_data, gen_kwargs)

print(f"Average ROUGE-1: Precision: {avg_rouge1['precision']:.4f}, Recall: {avg_rouge1['recall']:.4f}, F1 Score: {avg_rouge1['fmeasure']:.4f}")
print(f"Average ROUGE-2: Precision: {avg_rouge2['precision']:.4f}, Recall: {avg_rouge2['recall']:.4f}, F1 Score: {avg_rouge2['fmeasure']:.4f}")
print(f"Average ROUGE-L: Precision: {avg_rougeL['precision']:.4f}, Recall: {avg_rougeL['recall']:.4f}, F1 Score: {avg_rougeL['fmeasure']:.4f}")

Average ROUGE-1: Precision: 0.6153, Recall: 0.4771, F1 Score: 0.5251
Average ROUGE-2: Precision: 0.3027, Recall: 0.2356, F1 Score: 0.2601
Average ROUGE-L: Precision: 0.5083, Recall: 0.3957, F1 Score: 0.4341
